# 上傳資料集與模型到 HuggingFace Hub

此 notebook 會上傳：
- 📊 資料集 (含 Dataset Card)
- 🤖 模型 (含 Model Card)

In [ ]:
!pip install datasets huggingface_hub transformers -q

In [ ]:
import json
from datasets import Dataset
from huggingface_hub import login, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer

# 登入 HuggingFace (需要 write 權限的 token)
# https://huggingface.co/settings/tokens
login()

In [ ]:
# ⚠️ 請修改以下設定
HF_USERNAME = "your-username"  # 你的 HuggingFace 用戶名

# 路徑設定
DATASET_PATH = "../data/coffee_order_zhtw.jsonl"
MODEL_PATH = "../outputs/qwen3_coffee_sft"
MODEL_CARD_PATH = "../docs/MODEL_CARD.md"
DATASET_CARD_PATH = "../docs/DATASET_CARD.md"

# Hub repo 名稱
DATASET_REPO = f"{HF_USERNAME}/coffee-order-zhtw"
MODEL_REPO = f"{HF_USERNAME}/qwen3-1.7b-coffee-sft"

# 是否設為私有
PRIVATE = False

## 上傳資料集

In [ ]:
# 載入 JSONL 資料集
def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line.strip()))
    return data

raw_data = load_jsonl(DATASET_PATH)
dataset = Dataset.from_list(raw_data)
print(f"載入 {len(dataset)} 筆資料")
print(dataset)

In [ ]:
# 讀取 Dataset Card
with open(DATASET_CARD_PATH, "r", encoding="utf-8") as f:
    dataset_card_content = f.read()

print("Dataset Card 預覽：")
print(dataset_card_content[:500] + "...")

In [ ]:
# 上傳資料集
print(f"📤 上傳資料集到 {DATASET_REPO}...")
dataset.push_to_hub(
    DATASET_REPO,
    private=PRIVATE,
    commit_message="Upload coffee order dataset (Traditional Chinese)"
)
print(f"✅ 資料集已上傳: https://huggingface.co/datasets/{DATASET_REPO}")

In [ ]:
# 上傳 Dataset Card (README.md)
api = HfApi()

print(f"📤 上傳 Dataset Card...")
api.upload_file(
    path_or_fileobj=DATASET_CARD_PATH,
    path_in_repo="README.md",
    repo_id=DATASET_REPO,
    repo_type="dataset",
    commit_message="Add dataset card"
)
print(f"✅ Dataset Card 已上傳")

## 上傳模型

In [ ]:
# 載入訓練好的模型
print(f"載入模型: {MODEL_PATH}")
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
print("✅ 模型載入完成")

In [ ]:
# 讀取 Model Card
with open(MODEL_CARD_PATH, "r", encoding="utf-8") as f:
    model_card_content = f.read()

print("Model Card 預覽：")
print(model_card_content[:500] + "...")

In [ ]:
# 上傳模型
print(f"📤 上傳模型到 {MODEL_REPO}...")
model.push_to_hub(
    MODEL_REPO,
    private=PRIVATE,
    commit_message="Upload fine-tuned Qwen3-1.7B coffee order model"
)
tokenizer.push_to_hub(
    MODEL_REPO,
    private=PRIVATE,
    commit_message="Upload tokenizer"
)
print(f"✅ 模型已上傳: https://huggingface.co/{MODEL_REPO}")

In [ ]:
# 上傳 Model Card (README.md)
print(f"📤 上傳 Model Card...")
api.upload_file(
    path_or_fileobj=MODEL_CARD_PATH,
    path_in_repo="README.md",
    repo_id=MODEL_REPO,
    repo_type="model",
    commit_message="Add model card"
)
print(f"✅ Model Card 已上傳")

In [ ]:
# 完成
print("\n" + "="*50)
print("🎉 上傳完成！")
print(f"📊 資料集: https://huggingface.co/datasets/{DATASET_REPO}")
print(f"🤖 模型: https://huggingface.co/{MODEL_REPO}")
print("="*50)